## Load dataframe and create a list of unique CEPs

### Logic:
- Extract a list of CEPs (Brazilian postal code) and remove duplicates
- Lookup db to check if a record already exists
- if yes, continue to next CEP
- if Not, scrap localities from 3rd party API. 
- save a new record in db

- TODO:
 - implement a path search to list all BDO reports available on folder 'reports'

### Previous functions were isolated to make this code more clean

First, import csv_loader. This function will load a csv file located at the "bdo_csv" folder and return a list of unique CEPs.

In [1]:
from utils.csv_manager import csv_loader
caminho_do_csv = '../bdo_csv/bdo_report_16-02-2021-16-55-00.csv'
ceps_csv = csv_loader(caminho_do_csv)

Saving a csv file localy. It will be used to populate de db and, if needed, to extract other statistics

In [2]:
ceps_csv.to_csv('../data/ceps_sem_localidades.csv', sep=',', header=['CEP'], index=False)

### Data cleaning finished. Now, start to populate the db

Import db_manager. This function will create a database and tables if not exists. After, will check for CEPs already in the database and, if not, will query an API for localities of this CEP.

In [ ]:
from utils import db_manager
db_manager.cep_loader()

SQLite version: ('3.31.1',)
Inserindo registro, CEP: 78053378, Localidade: Morada do Ouro
Inserindo registro, CEP: 78135301, Localidade: Canelas
Inserindo registro, CEP: 78110735, Localidade: 23 de Setembro
Inserindo registro, CEP: 78110738, Localidade: 23 de Setembro
Inserindo registro, CEP: 78135315, Localidade: Canelas
Inserindo registro, CEP: 78725140, Localidade: Jardim Primavera II
Inserindo registro, CEP: 78089610, Localidade: Residencial Coxipó
Inserindo registro, CEP: 78045220, Localidade: Cep nao encontrado
Inserindo registro, CEP: 78110760, Localidade: 23 de Setembro
Inserindo registro, CEP: 78135340, Localidade: Canelas
Inserindo registro, CEP: 78550400, Localidade: Jardim Primavera
Inserindo registro, CEP: 78053424, Localidade: Morada do Ouro
Inserindo registro, CEP: 78110768, Localidade: 23 de Setembro
Inserindo registro, CEP: 78089614, Localidade: Residencial Coxipó
Inserindo registro, CEP: 78053428, Localidade: Morada do Ouro
Inserindo registro, CEP: 78110773, Localidad

Inserindo registro, CEP: 78553381, Localidade: Comunidade Betel
Inserindo registro, CEP: 78135591, Localidade: Centro-Sul
Inserindo registro, CEP: 78135593, Localidade: Centro-Sul
Inserindo registro, CEP: 78053675, Localidade: Tancredo Neves
Inserindo registro, CEP: 78070060, Localidade: Jardim Petrópolis
Inserindo registro, CEP: 78053678, Localidade: Tancredo Neves
Inserindo registro, CEP: 78053680, Localidade: Tancredo Neves
Inserindo registro, CEP: 78750000, Localidade: Jardim das Paineiras
Inserindo registro, CEP: 78070068, Localidade: Jardim Petrópolis
Inserindo registro, CEP: 78070070, Localidade: Jardim Petrópolis
Inserindo registro, CEP: 78094652, Localidade: Parque Atalaia
Inserindo registro, CEP: 78070078, Localidade: Jardim Petrópolis
Inserindo registro, CEP: 78094654, Localidade: Parque Atalaia
Inserindo registro, CEP: 78094656, Localidade: Parque Atalaia
Inserindo registro, CEP: 78152000, Localidade: São Matheus
Inserindo registro, CEP: 78094658, Localidade: Parque Atalaia

Inserindo registro, CEP: 78717455, Localidade: Jardim Residencial Carlos Bezerra I
Inserindo registro, CEP: 78144016, Localidade: Petrópolis
Inserindo registro, CEP: 78070290, Localidade: Jardim Califórnia
Inserindo registro, CEP: 78144018, Localidade: Petrópolis
Inserindo registro, CEP: 78099481, Localidade: Jardim Industriário
Inserindo registro, CEP: 78750240, Localidade: Parque Residencial Universitário
Inserindo registro, CEP: 78144032, Localidade: Petrópolis
Inserindo registro, CEP: 78144034, Localidade: Petrópolis
Inserindo registro, CEP: 78553643, Localidade: Jardim dos Ipês
Inserindo registro, CEP: 78070316, Localidade: Jardim Califórnia
Inserindo registro, CEP: 78070320, Localidade: Jardim Califórnia
Inserindo registro, CEP: 78553649, Localidade: Jardim dos Ipês
Inserindo registro, CEP: 78553653, Localidade: Jardim dos Ipês
Inserindo registro, CEP: 78717500, Localidade: Jardim Residencial Carlos Bezerra II
Inserindo registro, CEP: 78717505, Localidade: Jardim Residencial Carl

In [4]:
from pycep_correios import get_address_from_cep, WebService
address = get_address_from_cep(str(78736306), webservice=WebService.CORREIOS)

In [5]:
print(address)

{'bairro': 'Loteamento Residencial Alfredo de Castro Araújo', 'cep': '78736306', 'cidade': 'Rondonópolis', 'logradouro': 'Rua Vinte e Oito', 'uf': 'MT', 'complemento': None}


In [4]:
import sqlite3
import json
import codecs

In [5]:
import sqlite3
import pandas as pd
# Create your connection.
cnx = sqlite3.connect('../data/cepsdb.db')

df = pd.read_sql_query("SELECT * FROM Pareamento", cnx)

In [16]:
for cep in df.head(10)['cep']:
    
    string = str(cep)
    for i in string:
        print(ord(i))

98
39
92
120
48
50
92
120
48
48
92
120
97
55
92
120
48
52
92
120
48
48
92
120
48
48
92
120
48
48
92
120
48
48
39
98
39
92
120
48
53
64
92
120
97
56
92
120
48
52
92
120
48
48
92
120
48
48
92
120
48
48
92
120
48
48
39
98
39
92
120
48
102
92
120
101
48
92
120
97
55
92
120
48
52
92
120
48
48
92
120
48
48
92
120
48
48
92
120
48
48
39
98
39
92
120
49
50
92
120
101
48
92
120
97
55
92
120
48
52
92
120
48
48
92
120
48
48
92
120
48
48
92
120
48
48
39
98
39
92
120
49
51
64
92
120
97
56
92
120
48
52
92
120
48
48
92
120
48
48
92
120
48
48
92
120
48
48
39
98
39
92
120
49
52
64
92
120
98
49
92
120
48
52
92
120
48
48
92
120
48
48
92
120
48
48
92
120
48
48
39
98
39
92
120
56
97
92
120
56
100
92
120
97
55
92
120
48
52
92
120
48
48
92
120
48
48
92
120
48
48
92
120
48
48
39
98
39
36
92
120
101
48
92
120
97
54
92
120
48
52
92
120
48
48
92
120
48
48
92
120
48
48
92
120
48
48
39
98
39
40
92
120
101
48
92
120
97
55
92
120
48
52
92
120
48
48
92
120
48
48
92
120
48
48
92
120
48
48
39
98
39
44
64
92
120
97
56
92

## Retrieving DB data to join with BDO report 

In [ ]:
caminho_do_csv = '../bdo_csv/bdo_report_16-02-2021-16-55-00.csv'

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
#fhand = codecs.open('where.js','w', "utf-8")
#fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1])
    try: js = json.loads(str(data))
    except: continue

    if not('status' in js and js['status'] == 'OK') : continue

    lat = js["results"][0]["geometry"]["location"]["lat"]
    lng = js["results"][0]["geometry"]["location"]["lng"]
    if lat == 0 or lng == 0 : continue
    where = js['results'][0]['formatted_address']
    where = where.replace("'","")
    try :
        print where, lat, lng

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue

fhand.write("\n];\n")
cur.close()
fhand.close()
print count, "records written to where.js"
print "Open where.html to view the data in a browser"
